In [4]:
import os
import sys
import time
import traceback
import matplotlib.pyplot as plt
from datetime import datetime
import warnings

from scipy import signal
from scipy.stats import zscore

import numpy as np
import pandas as pd

import shutil
import mne
import keras
# from ecgdetectors import Detectors

DATA_DIR = "G:/cap-sleep-database-1.0.0/cap-sleep-database-1.0.0/"
TARGET_HZ = 64
warnings.filterwarnings('ignore')

2023-08-15 02:30:48.911271: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 02:30:48.969224: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 02:30:48.969932: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 02:30:49.791642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ed/30/310fee0477ce46f722c561dd7e21eebca0d1d29bdb3cf4a2335b845fbba4/tensorflow-2.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Obtaining dependency information for flatbuffers>=23.1.21 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.8 MB/s eta 0:00:00
  Obtaining dependency information for grpcio<2.0,>=1.24.3 from https://files.pythonhosted.org/packages/f6/8c/6bba418b0b10eeaaaa5d1f2b86a6324549108661a5bb6400045b58eeab26/grpcio-1.57.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for keras<2.14,>=2.13.1 from https://files.pythonhosted.org/p

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 2.4 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.2/524.2 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:06m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 2.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 2.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 2.3 MB/s eta 0:00:002.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 2.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 2.9 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 2.7 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 2.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall

In [2]:
def load_cap_ecg_data_mne(edf_file, fs_target=64, log=print):
    try:
        raw = mne.io.read_raw_edf(edf_file, preload=False)
        ch_idx = -1
        ch_name = None
        for cname in raw.info.get('ch_names'):
            ch_idx += 1
            if cname.find("ECG") > -1:
                ch_name = cname
                break
        hz = mne.pick_info(raw.info, [ch_idx], verbose=False)['sfreq']
        hz = int(hz)
        raw.pick_channels([ch_name])
        recording = raw.get_data().flatten()
        print(f"channel: {cname}")

    except:
        log(f"Error reading {edf_file}, caused by - {traceback.format_exc()}")
        return
    meta = {
        "hz": hz,
        "dt_signal_start": raw.info.get('meas_date')
    }
    # Down sample to 50Hz
    down_factor = meta["hz"] // fs_target
    target_samples = len(recording) // down_factor
    print(f"{edf_file}, @{meta['hz']}, down_factor:{down_factor}, recording:{recording.shape}, target:{target_samples}")
    recording = signal.resample(recording, target_samples)

    # Read annotation
    labels = []
    return (recording, labels, meta)

In [3]:
def generate_rr_signal(ecg_signal):
    rr_signal = np.zeros((ecg_signal.shape[0]))
    r_peaks = ECG_DETECTOR.pan_tompkins_detector(ecg_signal)
    r_peak_diff = np.diff(r_peaks)
    rr_signal[0:len(r_peak_diff)] = r_peak_diff
    return (r_peaks, rr_signal)

In [4]:
def sample_offset_stm(stm_annot, stm_signal, target_hz=64):
    print(f"tm_annot:{stm_annot}, tm_signal:{stm_signal}, target_hz:{target_hz}")
    tm_annot = datetime.strptime(stm_annot, "%H:%M:%S")
    tm_signal = datetime.strptime(stm_signal, "%H:%M:%S")
    delta_sec = int((tm_annot-tm_signal).total_seconds())
    return delta_sec*target_hz

In [5]:
def stm_annot_start(annot_filepath):
    with open(annot_filepath, "r") as f:
        lines = f.readlines()
    reached_sleep_stage_line = False    
    for line in lines:
        # print(line)
        tokens = line.split("\t")
        if not reached_sleep_stage_line and not line.startswith("Sleep Stage"):
            continue
        if line.startswith("Sleep Stage"):
            reached_sleep_stage_line = True
            continue
        # first annotation line
        s_time = tokens[-4]
        if s_time.find(":") < 0:
            print(f"Invalid timestamp | {line}")
            return None
        break
    return s_time

In [2]:
def edf_to_csv(data_directory):
    # Find and read .edf files and corresponding .txt files for annotation.
    #
    count_file = 0
    ignore_file = 0
    bad_files = []
    record_names = []
    target_data_dir = f"{data_directory}/np_data_{TARGET_HZ}"
    if not os.path.exists(target_data_dir):
        os.makedirs(target_data_dir)

    for f in os.listdir(data_directory):
        if not f.endswith(".edf"):
            continue
        rec_name = f[:-4]  # exclude .edf
        try:
            edf_path = f"{data_directory}/{f}"
            recording, _, meta = load_cap_ecg_data_mne(edf_path, fs_target=TARGET_HZ)
            
            # Annotation does not start from the beginning of a singal, 
            # discard un-annotated initial signal samples.
            # 
            stm_annot_begin = stm_annot_start(edf_path.replace('.edf', '.txt'))

            n_samples_skip = sample_offset_stm(stm_annot_begin, meta["dt_signal_start"].time().strftime("%H:%M:%S"))            
            
            print(
                f"{count_file} | {f} | signal:{recording.shape}, "
                f"offset:{n_samples_skip} samples ({stm_annot_begin}-{meta['dt_signal_start'].time().strftime('%H:%M:%S')}), "
                f"signal*:{len(recording)-n_samples_skip}")
            len_before = len(recording)
            recording = recording[n_samples_skip:]
            assert len(recording) == len_before - n_samples_skip

            record_names.append(rec_name)

            np.savetxt(f"{target_data_dir}/{f.replace('.edf', '.csv')}", recording)
            shutil.copy(f"{data_directory}/{f.replace('.edf', '.txt')}", target_data_dir)

            # rr_signal = generate_rr_signal(recording)
            # print(f"rr_signal")
            # break
        except:
            ignore_file += 1
            print(f"error loading {f}")
            traceback.print_exc(file=sys.stdout)
            bad_files.append(f)
            # Ignore corrupted recording.
            continue
        count_file += 1    
        # break
    print(f"{count_file} files created.")

In [7]:
# edf_to_csv(DATA_DIR)

In [2]:
class DataGenerator(keras.utils.Sequence):
    def __init__(
            self, np_data_dir, list_rec_names=None, batch_sz=32, hz=64, n_channels=1, shuffle=True, log=print,
            class_map={
                "S0": 0,
                "S1": 1,
                "S2": 1,
                "S3": 1,
                "S4": 1,
                "REM": 1,
            }
    ):
        self.np_data_dir = np_data_dir
        self.list_rec_names = list_rec_names
        self.batch_sz = batch_sz
        self.hz = hz
        self.class_map = class_map
        self.n_classes = 2
        self.dim = self.hz * 30  # 30 sec @64Hz
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.log = log
        self.samples_meta = []
        self.cached_data={}

        self._init_data_dict()

    def _init_data_dict(self):
        self.log("Initialise meta data of samples")
        reached_sleep_stage_line = False
        for f in os.listdir(self.np_data_dir):
            reached_sleep_stage_line = False
            seg_count = 0
            if not f.endswith(".csv"):
                continue
            rec_name = f[:-4]
            if not rec_name in self.list_rec_names:
                continue            
            self.log(f"Loading {rec_name}...")
            sig_filepath = f"{self.np_data_dir}/{f}"
            ecg_sig = np.loadtxt(sig_filepath, delimiter=",")
            annot_filepath = sig_filepath.replace('.csv', '.txt')
            with open(annot_filepath, "r") as f:
                lines = f.readlines()
            n_seg_est = ecg_sig.shape[0]//self.dim    
            self.log(f"[{rec_name}] {ecg_sig.shape}, n_seg:{n_seg_est}")
            start = 0
            ignored_annots_bad_len, ignored_annots_bad_label = 0, 0
            self.log(f"recording:{ecg_sig.shape}")
            for line in lines:
                # print(line)
                tokens = line.split("\t")
                if not reached_sleep_stage_line and not line.startswith("Sleep Stage"):
                    continue
                if line.startswith("Sleep Stage"):
                    reached_sleep_stage_line = True
                    continue
                # Annotation starts
                try:
                    tok_is_sleep = tokens[-3]
                    if tok_is_sleep.find("SLEEP-") < 0:
                        current_epoch_len = int(tokens[-2])
                        continue
                    label_str = tok_is_sleep.split("-")[-1]
                    label = self.class_map.get(label_str)
                    current_epoch_len = int(tokens[-2])
                except:
                    self.log(f"--> error parsing epoch-len: {tokens[-2]}")
                    ignored_annots_bad_len += 1
                    continue
                if current_epoch_len != 30:  # not a valid sleep epoch
                    ignored_annots_bad_len += 1
                    self.log(f"--> bad length: {current_epoch_len}")
                    # start += (current_epoch_len*self.hz)
                    continue
                if label is None:
                    self.log(
                        f"Ignored label for label_str:{label_str}->{label} >> {line}")
                    ignored_annots_bad_label += 1
                    start += self.dim
                    continue
                end = start + self.dim
                seg = ecg_sig[start : end]
                remaining = len(ecg_sig) - end

                if len(seg) != self.dim:
                    self.log(f"bad seg_sz:{len(seg)} >> {line}")
                    start += self.dim
                    continue

                self.samples_meta.append((rec_name, start, label))    

                # update start
                start += self.dim
                seg_count += 1    

            self.log(
                f"...[{rec_name}], n_seg:{seg_count} out of {n_seg_est} (lags seg:{n_seg_est-seg_count}, "
                f"samples:{(n_seg_est-seg_count)*self.dim}), bad_label: {ignored_annots_bad_label}, bad_len/ignored: {ignored_annots_bad_len}, "
                f"remaining:{remaining}")
        self.log(f"Total sample-meta loaded: {len(self.samples_meta)}")
#         self.indexes = range(len(self.samples_meta))
        self.indexes = list(range(len(self.samples_meta)))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.indexes) / self.batch_sz))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_sz:(index+1)*self.batch_sz]
        X, y = self.__data_generation(batch_indexes)
        return X, y
    
    def on_epoch_end(self):
        np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_sz, self.dim, self.n_channels))
        y = np.empty((self.batch_sz), dtype=int)

        # Generate data, retrieve from files
        for i, ID in enumerate(list_IDs_temp):
            self.log(f"i:{i} => {self.samples_meta[self.indexes[i]]} | ID:{ID} => {self.samples_meta[self.indexes[ID]]}")
            rec_name, seg_start, label = self.samples_meta[self.indexes[ID]]
            cached_key=rec_name+str(seg_start)
            if(self.cached_data.get(cached_key) is not None):
                segment=self.cached_data.get(cached_key)
            else:                
                # sig_filepath = os.path.join(self.data_dir, "np_data_64", f"{rec_name}.csv")
                sig_filepath = os.path.join(self.np_data_dir, f"{rec_name}.csv")
                ecg_sig = np.loadtxt(sig_filepath, delimiter=",")
                segment = ecg_sig[seg_start:seg_start+self.dim]
                segment = zscore(segment)  # normalise segment
                self.cached_data[cached_key]=segment
                
            X[i,] = segment.reshape((self.dim, 1))
            y[i] = label
        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
    
    def get_labels(self):
        labels = []
        for i in self.indexes:
            _, _, label = self.samples_meta[self.indexes[i]]
            labels.append(label)
        return np.array(labels)

In [10]:
training_generator = DataGenerator(np_data_dir="G:/cap-sleep-database-1.0.0/cap-sleep-database-1.0.0/np_data_64", list_rec_names=["n2", "n3"])

Initialise meta data of samples
Loading n2...
[n2] (2104320,), n_seg:1096
recording:(2104320,)
...[n2], n_seg:999 out of 1096 (lags seg:97, samples:186240), bad_label: 0, bad_len/ignored: 0, remaining:186240
Loading n3...
[n3] (1921984,), n_seg:1001
recording:(1921984,)
...[n3], n_seg:999 out of 1001 (lags seg:2, samples:3840), bad_label: 0, bad_len/ignored: 0, remaining:3904
Total sample-meta loaded: 1998


In [19]:
validation_generator = DataGenerator(np_data_dir="G:/cap-sleep-database-1.0.0/cap-sleep-database-1.0.0/np_data_64", list_rec_names=["n1"])

Initialise meta data of samples
Loading n3...
[n3] (1921984,), n_seg:1001
recording:(1921984,)
...[n3], n_seg:999 out of 1001 (lags seg:2, samples:3840), bad_label: 0, bad_len/ignored: 0, remaining:3904
Total sample-meta loaded: 999


In [21]:
print("number of batches:", len(validation_generator))


number of batches: 31


In [9]:
training_generator = DataGenerator(np_data_dir="G:/cap-sleep-database-1.0.0/cap-sleep-database-1.0.0/np_data_64", list_rec_names=["n2", "n3"])

for i, name in enumerate(training_generator.list_rec_names):
    batch_X, batch_y = training_generator.__getitem__(i, 0)    


Initialise meta data of samples
Loading n2...
[n2] (2104320,), n_seg:1096
recording:(2104320,)
...[n2], n_seg:999 out of 1096 (lags seg:97, samples:186240), bad_label: 0, bad_len/ignored: 0, remaining:186240
Loading n3...
[n3] (1921984,), n_seg:1001
recording:(1921984,)
...[n3], n_seg:999 out of 1001 (lags seg:2, samples:3840), bad_label: 0, bad_len/ignored: 0, remaining:3904
Total sample-meta loaded: 1998


TypeError: __data_generation() takes 2 positional arguments but 3 were given

In [30]:

batch_x_train, batch_y_train, batch_x_val, batch_y_val = leave_ones(0)

Initialise meta data of samples
Loading n2...
[n2] (2104320,), n_seg:1096
recording:(2104320,)
...[n2], n_seg:999 out of 1096 (lags seg:97, samples:186240), bad_label: 0, bad_len/ignored: 0, remaining:186240
Total sample-meta loaded: 999
Number of batches for training generator: 31
i:0 => ('n2', 0, 0) | ID:0 => ('n2', 0, 0)
i:1 => ('n2', 1920, 0) | ID:1 => ('n2', 1920, 0)
i:2 => ('n2', 3840, 0) | ID:2 => ('n2', 3840, 0)
i:3 => ('n2', 5760, 0) | ID:3 => ('n2', 5760, 0)
i:4 => ('n2', 7680, 0) | ID:4 => ('n2', 7680, 0)
i:5 => ('n2', 9600, 0) | ID:5 => ('n2', 9600, 0)
i:6 => ('n2', 11520, 0) | ID:6 => ('n2', 11520, 0)
i:7 => ('n2', 13440, 0) | ID:7 => ('n2', 13440, 0)
i:8 => ('n2', 15360, 1) | ID:8 => ('n2', 15360, 1)
i:9 => ('n2', 17280, 1) | ID:9 => ('n2', 17280, 1)
i:10 => ('n2', 19200, 1) | ID:10 => ('n2', 19200, 1)
i:11 => ('n2', 21120, 1) | ID:11 => ('n2', 21120, 1)
i:12 => ('n2', 23040, 1) | ID:12 => ('n2', 23040, 1)
i:13 => ('n2', 24960, 1) | ID:13 => ('n2', 24960, 1)
i:14 => ('n2'

In [40]:
print("number of batches:", len(training_generator))
batch_x, batch_y = training_generator[35] 
print("batch-data-shape:", batch_x.shape, ", batch_size:", batch_x.shape[0], ", single-item-in-batch:", batch_x[0].shape)

number of batches: 66
i:0 => ('n1', 0, 0) | ID:1120 => ('n1', 2150400, 1)
i:4 => ('n1', 7680, 0) | ID:1764 => ('n2', 1198080, 1)
i:1 => ('n1', 1920, 0) | ID:1121 => ('n1', 2152320, 1)
i:5 => ('n1', 9600, 0) | ID:1765 => ('n2', 1200000, 1)
i:2 => ('n1', 3840, 0) | ID:1122 => ('n1', 2154240, 1)
i:6 => ('n1', 11520, 0) | ID:1766 => ('n2', 1201920, 1)
i:3 => ('n1', 5760, 0) | ID:1123 => ('n1', 2156160, 1)
i:7 => ('n1', 13440, 0) | ID:1767 => ('n2', 1203840, 1)
i:4 => ('n1', 7680, 0) | ID:1124 => ('n1', 2158080, 1)
i:8 => ('n1', 15360, 0) | ID:1768 => ('n2', 1205760, 1)
i:9 => ('n1', 17280, 0) | ID:1769 => ('n2', 1207680, 1)
i:5 => ('n1', 9600, 0) | ID:1125 => ('n1', 2160000, 1)
i:10 => ('n1', 19200, 0) | ID:1770 => ('n2', 1209600, 1)
i:6 => ('n1', 11520, 0) | ID:1126 => ('n1', 2161920, 1)
i:11 => ('n1', 21120, 1) | ID:1771 => ('n2', 1211520, 1)
i:7 => ('n1', 13440, 0) | ID:1127 => ('n1', 2163840, 1)
i:12 => ('n1', 23040, 1) | ID:1772 => ('n2', 1213440, 1)
i:8 => ('n1', 15360, 0) | ID:1128 

In [10]:
for i in range(batch_x.shape[0]):
    signal = batch_x[i].T[0]
    label = batch_y[i]
    print(signal.shape)
    plt.plot(range(len(signal)), signal)
    plt.title(f"#{i} {label}")
    plt.show()

NameError: name 'batch_x' is not defined

**Spliting**

**Model**

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D , Flatten

In [4]:
model=Sequential()
model.add(Conv1D(input_shape=(1920,1),filters=64,kernel_size=(21),strides=(5),activation="relu"))
model.add(Conv1D(filters=64,kernel_size=(7), activation="relu"))
model.add(MaxPool1D(pool_size=(2),strides=(2)))
model.add(Conv1D(filters=128,kernel_size=(5),activation="relu"))
model.add(Conv1D(filters=128,kernel_size=(5), activation="relu"))
model.add(MaxPool1D(pool_size=(2),strides=(2)))

In [5]:
model.add(Flatten())
model.add(Dense(units=1024,activation="relu"))
model.add(Dense(units=512,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [6]:
model.compile(optimizer="adam", loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 380, 64)           1408      
                                                                 
 conv1d_1 (Conv1D)           (None, 374, 64)           28736     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 187, 64)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 183, 128)          41088     
                                                                 
 conv1d_3 (Conv1D)           (None, 179, 128)          82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 89, 128)          0         
 1D)                                                    

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=5,generator=training_generator, validation_data= validation_generator, validation_steps=2,epochs=2,callbacks=[checkpoint,early])

i:0 => ('n2', 0, 0) | ID:0 => ('n2', 0, 0)
i:1 => ('n2', 1920, 0) | ID:1 => ('n2', 1920, 0)
i:2 => ('n2', 3840, 0) | ID:2 => ('n2', 3840, 0)
i:3 => ('n2', 5760, 0) | ID:3 => ('n2', 5760, 0)
i:4 => ('n2', 7680, 0) | ID:4 => ('n2', 7680, 0)
i:5 => ('n2', 9600, 0) | ID:5 => ('n2', 9600, 0)
i:6 => ('n2', 11520, 0) | ID:6 => ('n2', 11520, 0)
i:7 => ('n2', 13440, 0) | ID:7 => ('n2', 13440, 0)
i:8 => ('n2', 15360, 1) | ID:8 => ('n2', 15360, 1)
i:9 => ('n2', 17280, 1) | ID:9 => ('n2', 17280, 1)
i:10 => ('n2', 19200, 1) | ID:10 => ('n2', 19200, 1)
i:11 => ('n2', 21120, 1) | ID:11 => ('n2', 21120, 1)
i:12 => ('n2', 23040, 1) | ID:12 => ('n2', 23040, 1)
i:13 => ('n2', 24960, 1) | ID:13 => ('n2', 24960, 1)
i:14 => ('n2', 26880, 1) | ID:14 => ('n2', 26880, 1)
i:15 => ('n2', 28800, 1) | ID:15 => ('n2', 28800, 1)
i:16 => ('n2', 30720, 1) | ID:16 => ('n2', 30720, 1)
i:17 => ('n2', 32640, 1) | ID:17 => ('n2', 32640, 1)
i:18 => ('n2', 34560, 1) | ID:18 => ('n2', 34560, 1)
i:19 => ('n2', 36480, 1) | ID:1

i:15 => ('n2', 28800, 1) | ID:687 => ('n2', 1319040, 0)
i:16 => ('n2', 30720, 1) | ID:688 => ('n2', 1320960, 1)
i:17 => ('n2', 32640, 1) | ID:689 => ('n2', 1322880, 1)
i:18 => ('n2', 34560, 1) | ID:690 => ('n2', 1324800, 1)
i:19 => ('n2', 36480, 1) | ID:691 => ('n2', 1326720, 1)
i:20 => ('n2', 38400, 1) | ID:692 => ('n2', 1328640, 1)
i:21 => ('n2', 40320, 1) | ID:693 => ('n2', 1330560, 0)
i:22 => ('n2', 42240, 1) | ID:694 => ('n2', 1332480, 0)
i:23 => ('n2', 44160, 1) | ID:695 => ('n2', 1334400, 1)
i:24 => ('n2', 46080, 1) | ID:696 => ('n2', 1336320, 1)
i:25 => ('n2', 48000, 1) | ID:697 => ('n2', 1338240, 1)
i:26 => ('n2', 49920, 1) | ID:698 => ('n2', 1340160, 1)
i:27 => ('n2', 51840, 1) | ID:699 => ('n2', 1342080, 1)
i:28 => ('n2', 53760, 1) | ID:700 => ('n2', 1344000, 1)
i:29 => ('n2', 55680, 1) | ID:701 => ('n2', 1345920, 1)
i:30 => ('n2', 57600, 1) | ID:702 => ('n2', 1347840, 1)
i:31 => ('n2', 59520, 0) | ID:703 => ('n2', 1349760, 1)
i:0 => ('n2', 0, 0) | ID:864 => ('n2', 1658880, 

i:5 => ('n1', 9600, 0) | ID:37 => ('n1', 71040, 1)
i:6 => ('n1', 11520, 0) | ID:38 => ('n1', 72960, 1)
i:7 => ('n1', 13440, 0) | ID:39 => ('n1', 74880, 1)
i:8 => ('n1', 15360, 0) | ID:40 => ('n1', 76800, 1)
i:9 => ('n1', 17280, 0) | ID:41 => ('n1', 78720, 1)
i:10 => ('n1', 19200, 0) | ID:42 => ('n1', 80640, 1)
i:11 => ('n1', 21120, 1) | ID:43 => ('n1', 82560, 1)
i:12 => ('n1', 23040, 1) | ID:44 => ('n1', 84480, 1)
i:13 => ('n1', 24960, 1) | ID:45 => ('n1', 86400, 1)
i:14 => ('n1', 26880, 1) | ID:46 => ('n1', 88320, 1)
i:15 => ('n1', 28800, 1) | ID:47 => ('n1', 90240, 1)
i:16 => ('n1', 30720, 1) | ID:48 => ('n1', 92160, 1)
i:17 => ('n1', 32640, 1) | ID:49 => ('n1', 94080, 1)
i:18 => ('n1', 34560, 1) | ID:50 => ('n1', 96000, 1)
i:19 => ('n1', 36480, 1) | ID:51 => ('n1', 97920, 1)
i:20 => ('n1', 38400, 1) | ID:52 => ('n1', 99840, 1)
i:21 => ('n1', 40320, 1) | ID:53 => ('n1', 101760, 1)
i:22 => ('n1', 42240, 1) | ID:54 => ('n1', 103680, 1)
i:23 => ('n1', 44160, 1) | ID:55 => ('n1', 105600,

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
names = ["n1", "n2", "n3"]
np_data_dir="G:/cap-sleep-database-1.0.0/cap-sleep-database-1.0.0/np_data_64"

def leave_ones():    
    for i in range(len(names)):
        val = names[i]
        train = names[i+1:] + names[:i]  
        
        print(val,train)
        print("==========================================================")
        checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
        early = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto')
        training_generator = DataGenerator(np_data_dir, list_rec_names=train)
        print("Number of batches for training generator:", len(training_generator))
        
        print("\n")
        print("==========================================================")    
        print("\n")
        validation_generator = DataGenerator(np_data_dir, list_rec_names=[val])        
        print("Number of batches for validation generator:", len(validation_generator))
        hist = model.fit_generator(steps_per_epoch=5,generator=training_generator, validation_data= validation_generator, validation_steps=5,epochs=10,callbacks=[checkpoint,early])
        

In [ ]:
leave_ones()

n1 ['n2', 'n3']
Initialise meta data of samples
Loading n2...
[n2] (2104320,), n_seg:1096
recording:(2104320,)
...[n2], n_seg:999 out of 1096 (lags seg:97, samples:186240), bad_label: 0, bad_len/ignored: 0, remaining:186240
Loading n3...
[n3] (1921984,), n_seg:1001
recording:(1921984,)
...[n3], n_seg:999 out of 1001 (lags seg:2, samples:3840), bad_label: 0, bad_len/ignored: 0, remaining:3904
Total sample-meta loaded: 1998
Number of batches for training generator: 62




Initialise meta data of samples
Loading n1...
[n1] (2202240,), n_seg:1147
recording:(2202240,)
...[n1], n_seg:1140 out of 1147 (lags seg:7, samples:13440), bad_label: 0, bad_len/ignored: 0, remaining:13440
Total sample-meta loaded: 1140
Number of batches for validation generator: 35
i:0 => ('n2', 49920, 1) | ID:26 => ('n2', 1113600, 1)
i:1 => ('n2', 245760, 1) | ID:128 => ('n2', 1345920, 1)
i:2 => ('n2', 1065600, 1) | ID:555 => ('n2', 1516800, 1)
i:3 => ('n2', 55680, 1) | ID:29 => ('n3', 72960, 1)
i:4 => ('n2', 1019520,

In [12]:
for val in names:
    train = [i for i in names if i != val ]
    print(val,train)
    

n1 ['n2', 'n3']
n2 ['n1', 'n3']
n3 ['n1', 'n2']


In [9]:
leave_ones()

Initialise meta data of samples
Loading n2...
[n2] (2104320,), n_seg:1096
recording:(2104320,)
...[n2], n_seg:999 out of 1096 (lags seg:97, samples:186240), bad_label: 0, bad_len/ignored: 0, remaining:186240
Loading n3...
[n3] (1921984,), n_seg:1001
recording:(1921984,)
...[n3], n_seg:999 out of 1001 (lags seg:2, samples:3840), bad_label: 0, bad_len/ignored: 0, remaining:3904
Total sample-meta loaded: 1998
Number of batches for training generator: 62
Initialise meta data of samples
Loading n1...
[n1] (2202240,), n_seg:1147
recording:(2202240,)
...[n1], n_seg:1140 out of 1147 (lags seg:7, samples:13440), bad_label: 0, bad_len/ignored: 0, remaining:13440
Total sample-meta loaded: 1140
Number of batches for validation generator: 35
i:0 => ('n2', 0, 0) | ID:0 => ('n2', 0, 0)
i:1 => ('n2', 1920, 0) | ID:1 => ('n2', 1920, 0)
i:2 => ('n2', 3840, 0) | ID:2 => ('n2', 3840, 0)
i:3 => ('n2', 5760, 0) | ID:3 => ('n2', 5760, 0)
i:4 => ('n2', 7680, 0) | ID:4 => ('n2', 7680, 0)
i:5 => ('n2', 9600, 0)

i:3 => ('n2', 5760, 0) | ID:355 => ('n2', 681600, 1)
i:4 => ('n2', 7680, 0) | ID:356 => ('n2', 683520, 1)
i:5 => ('n2', 9600, 0) | ID:357 => ('n2', 685440, 1)
i:6 => ('n2', 11520, 0) | ID:358 => ('n2', 687360, 1)
i:7 => ('n2', 13440, 0) | ID:359 => ('n2', 689280, 1)
i:8 => ('n2', 15360, 1) | ID:360 => ('n2', 691200, 1)
i:9 => ('n2', 17280, 1) | ID:361 => ('n2', 693120, 1)
i:10 => ('n2', 19200, 1) | ID:362 => ('n2', 695040, 1)
i:11 => ('n2', 21120, 1) | ID:363 => ('n2', 696960, 1)
i:12 => ('n2', 23040, 1) | ID:364 => ('n2', 698880, 1)
i:13 => ('n2', 24960, 1) | ID:365 => ('n2', 700800, 1)
i:14 => ('n2', 26880, 1) | ID:366 => ('n2', 702720, 1)
i:15 => ('n2', 28800, 1) | ID:367 => ('n2', 704640, 1)
i:16 => ('n2', 30720, 1) | ID:368 => ('n2', 706560, 1)
i:17 => ('n2', 32640, 1) | ID:369 => ('n2', 708480, 1)
i:18 => ('n2', 34560, 1) | ID:370 => ('n2', 710400, 1)
i:19 => ('n2', 36480, 1) | ID:371 => ('n2', 712320, 1)
i:20 => ('n2', 38400, 1) | ID:372 => ('n2', 714240, 1)
i:21 => ('n2', 40320

i:1 => ('n1', 1920, 0) | ID:33 => ('n1', 63360, 1)
i:2 => ('n1', 3840, 0) | ID:34 => ('n1', 65280, 1)
i:3 => ('n1', 5760, 0) | ID:35 => ('n1', 67200, 1)
i:4 => ('n1', 7680, 0) | ID:36 => ('n1', 69120, 1)
i:5 => ('n1', 9600, 0) | ID:37 => ('n1', 71040, 1)
i:6 => ('n1', 11520, 0) | ID:38 => ('n1', 72960, 1)
i:7 => ('n1', 13440, 0) | ID:39 => ('n1', 74880, 1)
i:8 => ('n1', 15360, 0) | ID:40 => ('n1', 76800, 1)
i:9 => ('n1', 17280, 0) | ID:41 => ('n1', 78720, 1)
i:10 => ('n1', 19200, 0) | ID:42 => ('n1', 80640, 1)
i:11 => ('n1', 21120, 1) | ID:43 => ('n1', 82560, 1)
i:12 => ('n1', 23040, 1) | ID:44 => ('n1', 84480, 1)
i:13 => ('n1', 24960, 1) | ID:45 => ('n1', 86400, 1)
i:14 => ('n1', 26880, 1) | ID:46 => ('n1', 88320, 1)
i:15 => ('n1', 28800, 1) | ID:47 => ('n1', 90240, 1)
i:16 => ('n1', 30720, 1) | ID:48 => ('n1', 92160, 1)
i:17 => ('n1', 32640, 1) | ID:49 => ('n1', 94080, 1)
i:18 => ('n1', 34560, 1) | ID:50 => ('n1', 96000, 1)
i:19 => ('n1', 36480, 1) | ID:51 => ('n1', 97920, 1)
i:20 =>

i:11 => ('n3', 1916160, 0) | ID:1069 => ('n3', 1263360, 0)
i:12 => ('n2', 1503360, 1) | ID:669 => ('n3', 443520, 1)
i:13 => ('n3', 729600, 1) | ID:1923 => ('n3', 330240, 1)
i:14 => ('n2', 526080, 1) | ID:1873 => ('n3', 731520, 1)
i:15 => ('n3', 1795200, 1) | ID:1533 => ('n3', 1056000, 1)
i:16 => ('n2', 1130880, 1) | ID:1628 => ('n3', 468480, 1)
i:17 => ('n3', 1691520, 1) | ID:901 => ('n3', 1221120, 1)
i:18 => ('n3', 867840, 1) | ID:1272 => ('n3', 1326720, 0)
i:19 => ('n3', 5760, 1) | ID:1953 => ('n2', 1847040, 0)
i:20 => ('n2', 879360, 0) | ID:74 => ('n3', 666240, 1)
i:21 => ('n2', 1637760, 1) | ID:287 => ('n2', 1290240, 0)
i:22 => ('n2', 1658880, 1) | ID:717 => ('n2', 1409280, 1)
i:23 => ('n2', 716160, 1) | ID:1136 => ('n3', 837120, 1)
i:24 => ('n2', 487680, 1) | ID:966 => ('n3', 289920, 1)
i:25 => ('n2', 631680, 0) | ID:1563 => ('n2', 1342080, 1)
i:26 => ('n3', 1399680, 1) | ID:1114 => ('n2', 933120, 1)
i:27 => ('n3', 819840, 1) | ID:1194 => ('n3', 1324800, 0)
i:28 => ('n3', 209280, 

i:27 => ('n3', 819840, 1) | ID:677 => ('n2', 1234560, 1)
i:28 => ('n1', 643200, 1) | ID:335 => ('n1', 374400, 1)
i:28 => ('n3', 209280, 1) | ID:405 => ('n2', 1608960, 1)
i:29 => ('n1', 441600, 1) | ID:230 => ('n1', 817920, 1)
i:29 => ('n2', 1783680, 1) | ID:1969 => ('n3', 324480, 1)
i:30 => ('n1', 1920000, 1) | ID:1000 => ('n1', 854400, 1)
i:30 => ('n3', 1635840, 1) | ID:998 => ('n2', 167040, 1)
i:31 => ('n1', 368640, 1) | ID:192 => ('n1', 1317120, 1)
i:31 => ('n2', 1263360, 1) | ID:1916 => ('n2', 890880, 0)
i:0 => ('n1', 1447680, 1) | ID:587 => ('n1', 1518720, 1)
i:1 => ('n1', 44160, 1) | ID:904 => ('n1', 687360, 1)
i:2 => ('n1', 1641600, 1) | ID:594 => ('n1', 1071360, 1)
i:3 => ('n1', 725760, 1) | ID:580 => ('n1', 1685760, 1)
i:4 => ('n1', 1340160, 1) | ID:423 => ('n1', 1248000, 1)
i:5 => ('n1', 1751040, 1) | ID:373 => ('n1', 1338240, 1)
i:6 => ('n1', 149760, 1) | ID:1012 => ('n1', 1344000, 1)
i:7 => ('n1', 144000, 1) | ID:673 => ('n1', 858240, 1)
i:8 => ('n1', 1699200, 1) | ID:263 =

...

i:18 => ('n2', 351360, 1) | ID:107 => ('n2', 1071360, 1)
i:19 => ('n2', 1628160, 0) | ID:83 => ('n2', 693120, 1)
i:20 => ('n2', 1336320, 1) | ID:757 => ('n2', 1741440, 1)
i:21 => ('n2', 829440, 1) | ID:504 => ('n2', 750720, 1)
i:22 => ('n2', 551040, 0) | ID:89 => ('n2', 1290240, 0)
i:23 => ('n2', 297600, 1) | ID:373 => ('n2', 268800, 1)
i:24 => ('n2', 721920, 1) | ID:535 => ('n2', 541440, 1)
i:25 => ('n2', 370560, 1) | ID:355 => ('n2', 892800, 1)
i:26 => ('n2', 618240, 1) | ID:38 => ('n2', 554880, 0)
i:27 => ('n2', 1660800, 1) | ID:514 => ('n2', 929280, 1)
i:28 => ('n2', 128640, 1) | ID:10 => ('n2', 802560, 1)
i:29 => ('n2', 215040, 1) | ID:611 => ('n2', 1889280, 0)
i:30 => ('n2', 172800, 1) | ID:352 => ('n2', 666240, 1)
i:31 => ('n2', 840960, 0) | ID:954 => ('n2', 149760, 1)
5/5 [==============================] - 2677s 557s/step - loss: 0.3623 - accuracy: 0.9250 - val_loss: 0.3402 - val_accuracy: 0.8594
Initialise meta data of samples
Loading n1...
[n1] (2202240,), n_seg:1147
recordin

i:16 => ('n1', 30720, 1) | ID:1488 => ('n2', 668160, 1)
i:17 => ('n1', 32640, 1) | ID:1489 => ('n2', 670080, 1)
i:18 => ('n1', 34560, 1) | ID:1490 => ('n2', 672000, 1)
i:19 => ('n1', 36480, 1) | ID:1491 => ('n2', 673920, 1)
i:20 => ('n1', 38400, 1) | ID:1492 => ('n2', 675840, 1)
i:21 => ('n1', 40320, 1) | ID:1493 => ('n2', 677760, 1)
i:22 => ('n1', 42240, 1) | ID:1494 => ('n2', 679680, 1)
i:23 => ('n1', 44160, 1) | ID:1495 => ('n2', 681600, 1)
i:24 => ('n1', 46080, 1) | ID:1496 => ('n2', 683520, 1)
i:25 => ('n1', 48000, 1) | ID:1497 => ('n2', 685440, 1)
i:26 => ('n1', 49920, 1) | ID:1498 => ('n2', 687360, 1)
i:27 => ('n1', 51840, 1) | ID:1499 => ('n2', 689280, 1)
i:28 => ('n1', 53760, 1) | ID:1500 => ('n2', 691200, 1)
i:29 => ('n1', 55680, 1) | ID:1501 => ('n2', 693120, 1)
i:30 => ('n1', 57600, 1) | ID:1502 => ('n2', 695040, 1)
i:31 => ('n1', 59520, 1) | ID:1503 => ('n2', 696960, 1)
i:0 => ('n1', 0, 0) | ID:1632 => ('n2', 944640, 1)
3/5 [=================>............] - ETA: 8:02 - lo

i:30 => ('n1', 57600, 1) | ID:2078 => ('n2', 1800960, 0)
i:1 => ('n3', 1920, 0) | ID:33 => ('n3', 63360, 1)
i:31 => ('n1', 59520, 1) | ID:2079 => ('n2', 1802880, 0)
i:2 => ('n3', 3840, 1) | ID:34 => ('n3', 65280, 1)
i:3 => ('n3', 5760, 1) | ID:35 => ('n3', 67200, 1)
i:4 => ('n3', 7680, 1) | ID:36 => ('n3', 69120, 1)
i:5 => ('n3', 9600, 1) | ID:37 => ('n3', 71040, 1)
i:6 => ('n3', 11520, 1) | ID:38 => ('n3', 72960, 1)
i:7 => ('n3', 13440, 1) | ID:39 => ('n3', 74880, 1)
i:8 => ('n3', 15360, 1) | ID:40 => ('n3', 76800, 1)
i:9 => ('n3', 17280, 0) | ID:41 => ('n3', 78720, 1)
i:10 => ('n3', 19200, 0) | ID:42 => ('n3', 80640, 1)
i:11 => ('n3', 21120, 0) | ID:43 => ('n3', 82560, 1)
i:12 => ('n3', 23040, 0) | ID:44 => ('n3', 84480, 1)
i:13 => ('n3', 24960, 0) | ID:45 => ('n3', 86400, 1)
i:14 => ('n3', 26880, 0) | ID:46 => ('n3', 88320, 1)
i:15 => ('n3', 28800, 0) | ID:47 => ('n3', 90240, 1)
i:16 => ('n3', 30720, 0) | ID:48 => ('n3', 92160, 1)
i:17 => ('n3', 32640, 1) | ID:49 => ('n3', 94080, 1)

i:9 => ('n2', 1557120, 1) | ID:497 => ('n2', 472320, 1)
i:10 => ('n1', 2004480, 1) | ID:261 => ('n2', 910080, 0)
i:11 => ('n1', 1418880, 1) | ID:1503 => ('n1', 1559040, 1)
i:12 => ('n2', 168960, 1) | ID:1210 => ('n2', 145920, 1)
i:13 => ('n2', 1305600, 1) | ID:83 => ('n1', 622080, 1)
i:14 => ('n1', 1614720, 1) | ID:660 => ('n2', 238080, 1)
i:15 => ('n1', 360960, 1) | ID:393 => ('n2', 624000, 1)
i:16 => ('n1', 2064000, 1) | ID:1165 => ('n1', 844800, 1)
i:17 => ('n1', 1762560, 1) | ID:996 => ('n2', 13440, 0)
i:18 => ('n1', 858240, 1) | ID:635 => ('n1', 464640, 0)
i:19 => ('n1', 935040, 1) | ID:275 => ('n2', 1883520, 0)
i:20 => ('n2', 984960, 1) | ID:1870 => ('n1', 1376640, 1)
i:21 => ('n1', 117120, 1) | ID:1090 => ('n2', 86400, 1)
i:22 => ('n1', 1240320, 1) | ID:1231 => ('n2', 1754880, 1)
i:23 => ('n2', 1653120, 1) | ID:424 => ('n1', 1056000, 1)
i:24 => ('n2', 1052160, 1) | ID:978 => ('n2', 1735680, 1)
i:25 => ('n1', 263040, 1) | ID:620 => ('n1', 1979520, 1)
i:26 => ('n1', 136320, 1) | I

i:26 => ('n3', 1307520, 0) | ID:681 => ('n3', 800640, 1)
i:27 => ('n2', 493440, 1) | ID:783 => ('n2', 1253760, 1)
i:27 => ('n3', 798720, 1) | ID:416 => ('n3', 506880, 1)
i:28 => ('n1', 1564800, 1) | ID:14 => ('n1', 1614720, 1)
i:28 => ('n3', 299520, 1) | ID:156 => ('n3', 1555200, 1)
i:29 => ('n2', 1912320, 0) | ID:1968 => ('n1', 1334400, 1)
i:29 => ('n3', 1522560, 1) | ID:793 => ('n3', 1701120, 1)
i:30 => ('n1', 875520, 1) | ID:357 => ('n1', 1921920, 1)
i:30 => ('n3', 428160, 1) | ID:223 => ('n3', 915840, 1)
i:31 => ('n1', 341760, 1) | ID:772 => ('n1', 312960, 1)
i:31 => ('n3', 230400, 1) | ID:120 => ('n3', 994560, 1)
i:0 => ('n3', 533760, 1) | ID:502 => ('n3', 347520, 1)
i:1 => ('n3', 445440, 1) | ID:789 => ('n3', 155520, 0)
i:2 => ('n3', 650880, 1) | ID:833 => ('n3', 1288320, 0)
i:3 => ('n3', 992640, 1) | ID:887 => ('n3', 1466880, 1)
i:4 => ('n3', 990720, 1) | ID:714 => ('n3', 288000, 1)
i:5 => ('n3', 1161600, 1) | ID:484 => ('n3', 616320, 1)
i:6 => ('n3', 1474560, 0) | ID:31 => ('n3

In [ ]:
print("Evaluate model on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

In [11]:
leave_ones()

Initialise meta data of samples
Loading n2...
[n2] (2104320,), n_seg:1096
recording:(2104320,)
...[n2], n_seg:999 out of 1096 (lags seg:97, samples:186240), bad_label: 0, bad_len/ignored: 0, remaining:186240
Loading n3...
[n3] (1921984,), n_seg:1001
recording:(1921984,)
...[n3], n_seg:999 out of 1001 (lags seg:2, samples:3840), bad_label: 0, bad_len/ignored: 0, remaining:3904
Total sample-meta loaded: 1998
Initialise meta data of samples
Loading n1...
[n1] (2202240,), n_seg:1147
recording:(2202240,)
...[n1], n_seg:1140 out of 1147 (lags seg:7, samples:13440), bad_label: 0, bad_len/ignored: 0, remaining:13440
Total sample-meta loaded: 1140
i:0 => ('n2', 0, 0) | ID:0 => ('n2', 0, 0)
i:1 => ('n2', 1920, 0) | ID:1 => ('n2', 1920, 0)
i:2 => ('n2', 3840, 0) | ID:2 => ('n2', 3840, 0)
i:3 => ('n2', 5760, 0) | ID:3 => ('n2', 5760, 0)
i:4 => ('n2', 7680, 0) | ID:4 => ('n2', 7680, 0)
i:5 => ('n2', 9600, 0) | ID:5 => ('n2', 9600, 0)
i:6 => ('n2', 11520, 0) | ID:6 => ('n2', 11520, 0)
i:7 => ('n2', 

i:4 => ('n2', 7680, 0) | ID:484 => ('n2', 929280, 1)
i:5 => ('n2', 9600, 0) | ID:485 => ('n2', 931200, 1)
i:6 => ('n2', 11520, 0) | ID:486 => ('n2', 933120, 1)
i:7 => ('n2', 13440, 0) | ID:487 => ('n2', 935040, 1)
i:8 => ('n2', 15360, 1) | ID:488 => ('n2', 936960, 1)
i:9 => ('n2', 17280, 1) | ID:489 => ('n2', 938880, 1)
i:10 => ('n2', 19200, 1) | ID:490 => ('n2', 940800, 1)
i:11 => ('n2', 21120, 1) | ID:491 => ('n2', 942720, 1)
i:12 => ('n2', 23040, 1) | ID:492 => ('n2', 944640, 1)
i:13 => ('n2', 24960, 1) | ID:493 => ('n2', 946560, 1)
i:14 => ('n2', 26880, 1) | ID:494 => ('n2', 948480, 1)
i:15 => ('n2', 28800, 1) | ID:495 => ('n2', 950400, 1)
i:16 => ('n2', 30720, 1) | ID:496 => ('n2', 952320, 1)
i:17 => ('n2', 32640, 1) | ID:497 => ('n2', 954240, 1)
i:18 => ('n2', 34560, 1) | ID:498 => ('n2', 956160, 1)
i:19 => ('n2', 36480, 1) | ID:499 => ('n2', 958080, 1)
i:20 => ('n2', 38400, 1) | ID:500 => ('n2', 960000, 1)
i:21 => ('n2', 40320, 1) | ID:501 => ('n2', 961920, 1)
i:22 => ('n2', 422

i:1 => ('n1', 1920, 0) | ID:33 => ('n1', 63360, 1)
i:2 => ('n1', 3840, 0) | ID:34 => ('n1', 65280, 1)
i:3 => ('n1', 5760, 0) | ID:35 => ('n1', 67200, 1)
i:4 => ('n1', 7680, 0) | ID:36 => ('n1', 69120, 1)
i:5 => ('n1', 9600, 0) | ID:37 => ('n1', 71040, 1)
i:6 => ('n1', 11520, 0) | ID:38 => ('n1', 72960, 1)
i:7 => ('n1', 13440, 0) | ID:39 => ('n1', 74880, 1)
i:8 => ('n1', 15360, 0) | ID:40 => ('n1', 76800, 1)
i:9 => ('n1', 17280, 0) | ID:41 => ('n1', 78720, 1)
i:10 => ('n1', 19200, 0) | ID:42 => ('n1', 80640, 1)
i:11 => ('n1', 21120, 1) | ID:43 => ('n1', 82560, 1)
i:12 => ('n1', 23040, 1) | ID:44 => ('n1', 84480, 1)
i:13 => ('n1', 24960, 1) | ID:45 => ('n1', 86400, 1)
i:14 => ('n1', 26880, 1) | ID:46 => ('n1', 88320, 1)
i:15 => ('n1', 28800, 1) | ID:47 => ('n1', 90240, 1)
i:16 => ('n1', 30720, 1) | ID:48 => ('n1', 92160, 1)
i:17 => ('n1', 32640, 1) | ID:49 => ('n1', 94080, 1)
i:18 => ('n1', 34560, 1) | ID:50 => ('n1', 96000, 1)
i:19 => ('n1', 36480, 1) | ID:51 => ('n1', 97920, 1)
i:20 =>

i:12 => ('n2', 1616640, 1) | ID:176 => ('n2', 472320, 1)
i:13 => ('n3', 483840, 1) | ID:771 => ('n2', 451200, 1)
i:14 => ('n2', 1731840, 1) | ID:1071 => ('n3', 1662720, 1)
i:15 => ('n2', 234240, 1) | ID:1931 => ('n3', 785280, 1)
i:16 => ('n2', 1152000, 1) | ID:281 => ('n3', 835200, 1)
i:17 => ('n2', 1461120, 1) | ID:1364 => ('n2', 994560, 1)
i:18 => ('n2', 1217280, 1) | ID:71 => ('n3', 1084800, 1)
i:19 => ('n3', 883200, 1) | ID:1819 => ('n2', 902400, 1)
i:20 => ('n3', 768000, 1) | ID:517 => ('n3', 1253760, 0)
i:21 => ('n3', 1854720, 1) | ID:1282 => ('n2', 696960, 1)
i:22 => ('n3', 1866240, 1) | ID:1182 => ('n3', 451200, 1)
i:23 => ('n2', 1618560, 1) | ID:374 => ('n2', 1605120, 1)
i:24 => ('n3', 1557120, 1) | ID:1205 => ('n3', 919680, 1)
i:25 => ('n2', 1641600, 1) | ID:465 => ('n3', 1898880, 1)
i:26 => ('n3', 393600, 1) | ID:1636 => ('n2', 1635840, 1)
i:27 => ('n2', 368640, 1) | ID:688 => ('n2', 53760, 1)
i:28 => ('n2', 1708800, 1) | ID:1107 => ('n2', 1294080, 0)
i:29 => ('n3', 1745280,

i:25 => ('n1', 1797120, 1) | ID:936 => ('n1', 1920000, 1)
i:31 => ('n3', 1380480, 1) | ID:1766 => ('n2', 604800, 1)
i:26 => ('n1', 1776000, 1) | ID:925 => ('n1', 19200, 0)
i:27 => ('n1', 1603200, 1) | ID:835 => ('n1', 821760, 1)
i:28 => ('n1', 1824000, 1) | ID:950 => ('n1', 1272960, 1)
i:29 => ('n1', 101760, 1) | ID:53 => ('n1', 1850880, 0)
i:30 => ('n1', 243840, 1) | ID:127 => ('n1', 1413120, 1)
i:31 => ('n1', 1543680, 1) | ID:804 => ('n1', 1866240, 1)
i:0 => ('n1', 435840, 1) | ID:64 => ('n1', 522240, 1)
i:1 => ('n1', 1474560, 1) | ID:174 => ('n1', 944640, 1)
i:2 => ('n1', 1511040, 1) | ID:735 => ('n1', 1357440, 1)
i:3 => ('n1', 2133120, 1) | ID:618 => ('n1', 1405440, 1)
i:4 => ('n1', 570240, 1) | ID:1073 => ('n1', 1697280, 1)
i:5 => ('n1', 1132800, 1) | ID:996 => ('n1', 946560, 1)
i:6 => ('n1', 1428480, 1) | ID:627 => ('n1', 589440, 1)
i:7 => ('n1', 600960, 1) | ID:119 => ('n1', 26880, 1)
i:8 => ('n1', 480000, 1) | ID:1120 => ('n1', 1182720, 1)
i:9 => ('n1', 2144640, 1) | ID:1034 =>

...

i:17 => ('n2', 228480, 1) | ID:809 => ('n2', 1219200, 1)
i:18 => ('n2', 1472640, 1) | ID:417 => ('n2', 510720, 1)
i:19 => ('n2', 1392000, 1) | ID:978 => ('n2', 1269120, 0)
i:20 => ('n2', 1505280, 1) | ID:968 => ('n2', 938880, 1)
i:21 => ('n2', 1069440, 1) | ID:505 => ('n2', 887040, 0)
i:22 => ('n2', 1311360, 1) | ID:660 => ('n2', 264960, 1)
i:23 => ('n2', 1432320, 1) | ID:506 => ('n2', 282240, 1)
i:24 => ('n2', 1104000, 1) | ID:168 => ('n2', 545280, 1)
i:25 => ('n2', 88320, 1) | ID:97 => ('n2', 1906560, 0)
i:26 => ('n2', 1036800, 1) | ID:647 => ('n2', 1171200, 1)
i:27 => ('n2', 1180800, 1) | ID:749 => ('n2', 1697280, 1)
i:28 => ('n2', 1539840, 1) | ID:32 => ('n2', 743040, 1)
i:29 => ('n2', 723840, 1) | ID:324 => ('n2', 136320, 1)
i:30 => ('n2', 1163520, 1) | ID:637 => ('n2', 1418880, 1)
i:31 => ('n2', 1378560, 1) | ID:141 => ('n2', 1752960, 1)
5/5 [==============================] - 1964s 438s/step - loss: 0.3423 - accuracy: 0.9250 - val_loss: 0.4329 - val_accuracy: 0.8594
Initialise me

i:15 => ('n1', 28800, 1) | ID:1039 => ('n1', 1994880, 1)
i:16 => ('n1', 30720, 1) | ID:1040 => ('n1', 1996800, 1)
i:17 => ('n1', 32640, 1) | ID:1041 => ('n1', 1998720, 1)
i:18 => ('n1', 34560, 1) | ID:1042 => ('n1', 2000640, 1)
i:19 => ('n1', 36480, 1) | ID:1043 => ('n1', 2002560, 1)
i:20 => ('n1', 38400, 1) | ID:1044 => ('n1', 2004480, 1)
i:21 => ('n1', 40320, 1) | ID:1045 => ('n1', 2006400, 1)
i:22 => ('n1', 42240, 1) | ID:1046 => ('n1', 2008320, 1)
i:23 => ('n1', 44160, 1) | ID:1047 => ('n1', 2010240, 1)
i:24 => ('n1', 46080, 1) | ID:1048 => ('n1', 2012160, 1)
i:25 => ('n1', 48000, 1) | ID:1049 => ('n1', 2014080, 1)
i:26 => ('n1', 49920, 1) | ID:1050 => ('n1', 2016000, 1)
i:27 => ('n1', 51840, 1) | ID:1051 => ('n1', 2017920, 1)
i:28 => ('n1', 53760, 1) | ID:1052 => ('n1', 2019840, 1)
i:29 => ('n1', 55680, 1) | ID:1053 => ('n1', 2021760, 1)
i:30 => ('n1', 57600, 1) | ID:1054 => ('n1', 2023680, 1)
i:31 => ('n1', 59520, 1) | ID:1055 => ('n1', 2025600, 1)
i:0 => ('n1', 0, 0) | ID:960 =>

i:28 => ('n1', 53760, 1) | ID:220 => ('n1', 422400, 1)
i:1 => ('n3', 1920, 0) | ID:33 => ('n3', 63360, 1)
i:29 => ('n1', 55680, 1) | ID:221 => ('n1', 424320, 1)
i:2 => ('n3', 3840, 1) | ID:34 => ('n3', 65280, 1)
i:30 => ('n1', 57600, 1) | ID:222 => ('n1', 426240, 1)
i:3 => ('n3', 5760, 1) | ID:35 => ('n3', 67200, 1)
i:31 => ('n1', 59520, 1) | ID:223 => ('n1', 428160, 1)
i:4 => ('n3', 7680, 1) | ID:36 => ('n3', 69120, 1)
i:5 => ('n3', 9600, 1) | ID:37 => ('n3', 71040, 1)
i:6 => ('n3', 11520, 1) | ID:38 => ('n3', 72960, 1)
i:7 => ('n3', 13440, 1) | ID:39 => ('n3', 74880, 1)
i:8 => ('n3', 15360, 1) | ID:40 => ('n3', 76800, 1)
i:9 => ('n3', 17280, 0) | ID:41 => ('n3', 78720, 1)
i:10 => ('n3', 19200, 0) | ID:42 => ('n3', 80640, 1)
i:11 => ('n3', 21120, 0) | ID:43 => ('n3', 82560, 1)
i:12 => ('n3', 23040, 0) | ID:44 => ('n3', 84480, 1)
i:13 => ('n3', 24960, 0) | ID:45 => ('n3', 86400, 1)
i:14 => ('n3', 26880, 0) | ID:46 => ('n3', 88320, 1)
i:15 => ('n3', 28800, 0) | ID:47 => ('n3', 90240, 1)

i:7 => ('n1', 1645440, 1) | ID:134 => ('n2', 155520, 1)
i:8 => ('n2', 1505280, 1) | ID:615 => ('n2', 1230720, 1)
i:9 => ('n1', 1914240, 1) | ID:160 => ('n2', 1317120, 1)
i:10 => ('n2', 1201920, 1) | ID:1410 => ('n2', 403200, 1)
i:11 => ('n1', 894720, 1) | ID:1671 => ('n2', 1117440, 1)
i:12 => ('n1', 2012160, 1) | ID:1421 => ('n2', 1338240, 1)
i:13 => ('n2', 1591680, 1) | ID:1400 => ('n1', 458880, 1)
i:14 => ('n1', 1608960, 1) | ID:85 => ('n2', 1384320, 1)
i:15 => ('n2', 904320, 1) | ID:646 => ('n1', 702720, 1)
i:16 => ('n1', 1042560, 1) | ID:1375 => ('n2', 685440, 1)
i:17 => ('n2', 1050240, 1) | ID:316 => ('n1', 153600, 1)
i:18 => ('n1', 1080960, 1) | ID:900 => ('n1', 326400, 1)
i:19 => ('n2', 17280, 1) | ID:105 => ('n1', 625920, 1)
i:20 => ('n1', 913920, 1) | ID:1105 => ('n1', 322560, 1)
i:21 => ('n1', 51840, 1) | ID:438 => ('n1', 309120, 1)
i:22 => ('n1', 1463040, 1) | ID:521 => ('n2', 1207680, 1)
i:23 => ('n1', 1893120, 1) | ID:1991 => ('n2', 422400, 1)
i:24 => ('n2', 666240, 1) | I

i:27 => ('n3', 19200, 0) | ID:10 => ('n3', 259200, 1)
i:25 => ('n1', 2104320, 1) | ID:1126 => ('n2', 510720, 1)
i:28 => ('n3', 631680, 1) | ID:329 => ('n3', 1409280, 1)
i:29 => ('n3', 902400, 0) | ID:470 => ('n3', 1589760, 1)
i:26 => ('n2', 1173120, 1) | ID:2000 => ('n1', 38400, 1)
i:27 => ('n1', 1228800, 1) | ID:1114 => ('n2', 533760, 1)
i:30 => ('n3', 55680, 1) | ID:29 => ('n3', 902400, 0)
i:28 => ('n2', 483840, 1) | ID:1009 => ('n1', 236160, 1)
i:31 => ('n3', 1420800, 1) | ID:740 => ('n3', 819840, 1)
i:29 => ('n2', 111360, 1) | ID:8 => ('n2', 1505280, 1)
i:0 => ('n3', 366720, 1) | ID:355 => ('n3', 1800960, 1)
i:30 => ('n2', 405120, 1) | ID:652 => ('n1', 833280, 1)
i:1 => ('n3', 5760, 1) | ID:194 => ('n3', 929280, 1)
i:31 => ('n1', 1119360, 1) | ID:1058 => ('n2', 1094400, 1)
i:2 => ('n3', 946560, 1) | ID:160 => ('n3', 1683840, 1)
i:3 => ('n3', 0, 0) | ID:302 => ('n3', 1818240, 1)
i:4 => ('n3', 1618560, 1) | ID:385 => ('n3', 182400, 1)
i:5 => ('n3', 1128960, 1) | ID:165 => ('n3', 1228

In [18]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
# , "n5", "n6", "n7", "n9", "n10", "n11", "n13","n14"
names = ["n1", "n2", "n3"]
np_data_dir="G:/cap-sleep-database-1.0.0/cap-sleep-database-1.0.0/np_data_64"

def leave_ones():
    metrics_df = pd.DataFrame(columns=['validation_sub', 'loss', 'accuracy', 'val_loss', 'val_accuracy', 'time_per_epoch'])
    
    for i in range(len(names)):
        val = names[i]
        train = names[i+1:] + names[:i]
        
        print(val,train)
        print("==========================================================")
        checkpoint = ModelCheckpoint(f"vgg16_1_{val}.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
        early = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, mode='auto')
        training_generator = DataGenerator(np_data_dir, list_rec_names=train)
        print("Number of batches for training generator:", len(training_generator))
        print("==========================================================")
        validation_generator = DataGenerator(np_data_dir, list_rec_names=[val])
        print("Number of batches for validation generator:", len(validation_generator))
        
        start_time = time.time()
        hist = model.fit_generator(steps_per_epoch=5, generator=training_generator, validation_data=validation_generator, validation_steps=5, epochs=30, callbacks=[checkpoint, early])
        end_time = time.time()
        
        iteration_name = f"Subject-{val}"
        hist_df = pd.DataFrame(hist.history)
        hist_df['Subject'] = iteration_name
        
        
        time_per_epoch = (end_time - start_time) / len(hist.history['loss'])
        hist_df['time_per_epoch'] = time_per_epoch
        
        metrics_df = metrics_df.append(hist_df, ignore_index=True)
        
        # Save the metrics DataFrame to Excel after each iteration (epoch)
        metrics_df.to_excel('metrics.xlsx', index=False)


In [ ]:
leave_ones()

n1 ['n2', 'n3', 'n5', 'n6', 'n7', 'n9', 'n10', 'n11', 'n13', 'n14']
Initialise meta data of samples
Loading n10...
[n10] (1651200,), n_seg:860
recording:(1651200,)
...[n10], n_seg:852 out of 860 (lags seg:8, samples:15360), bad_label: 0, bad_len/ignored: 0, remaining:15360
Loading n11...
[n11] (2021760,), n_seg:1053
recording:(2021760,)
...[n11], n_seg:1051 out of 1053 (lags seg:2, samples:3840), bad_label: 0, bad_len/ignored: 0, remaining:3840
Loading n13...
[n13] (1944520,), n_seg:1012
recording:(1944520,)
...[n13], n_seg:952 out of 1012 (lags seg:60, samples:115200), bad_label: 0, bad_len/ignored: 0, remaining:116680
Loading n14...
[n14] (1947026,), n_seg:1014
recording:(1947026,)
...[n14], n_seg:965 out of 1014 (lags seg:49, samples:94080), bad_label: 0, bad_len/ignored: 0, remaining:94226
Loading n2...
[n2] (2104320,), n_seg:1096
recording:(2104320,)
...[n2], n_seg:999 out of 1096 (lags seg:97, samples:186240), bad_label: 0, bad_len/ignored: 0, remaining:186240
Loading n3...
[n3] 

 2/10 [=====>........................] - ETA: 31:36 - loss: 0.7535 - accuracy: 0.5312i:1 => ('n11', 449280, 1) | ID:5798 => ('n13', 1605120, 0)
i:2 => ('n11', 1741440, 1) | ID:1564 => ('n2', 322560, 1)
i:3 => ('n5', 1841280, 1) | ID:3460 => ('n13', 1507200, 1)
i:4 => ('n6', 529920, 1) | ID:2228 => ('n10', 683520, 1)
i:5 => ('n2', 1516800, 1) | ID:2658 => ('n5', 282240, 1)
i:6 => ('n11', 368640, 1) | ID:8374 => ('n3', 938880, 1)
i:7 => ('n6', 1724160, 1) | ID:5960 => ('n10', 23040, 0)
i:8 => ('n2', 1555200, 1) | ID:2166 => ('n5', 1209600, 1)
i:9 => ('n13', 1476480, 1) | ID:3956 => ('n2', 74880, 1)
i:10 => ('n9', 1203840, 1) | ID:3023 => ('n11', 1507200, 1)
i:11 => ('n11', 84480, 0) | ID:2943 => ('n7', 1040640, 1)
i:12 => ('n14', 842880, 1) | ID:9665 => ('n9', 1787520, 0)
i:13 => ('n3', 34560, 1) | ID:6096 => ('n2', 232320, 1)
i:14 => ('n2', 120960, 1) | ID:1628 => ('n5', 243840, 1)
i:15 => ('n10', 1305600, 1) | ID:2165 => ('n6', 1144320, 1)
i:16 => ('n7', 1380480, 1) | ID:6962 => ('n13'

i:7 => ('n6', 1724160, 1) | ID:2492 => ('n14', 1486080, 1)
i:8 => ('n2', 1555200, 1) | ID:2386 => ('n3', 948480, 1)
i:9 => ('n13', 1476480, 1) | ID:187 => ('n5', 261120, 1)
i:10 => ('n9', 1203840, 1) | ID:5348 => ('n9', 1082880, 1)
i:11 => ('n11', 84480, 0) | ID:2082 => ('n6', 702720, 1)
i:12 => ('n14', 842880, 1) | ID:5760 => ('n9', 201600, 1)
i:13 => ('n3', 34560, 1) | ID:452 => ('n11', 971520, 1)
i:14 => ('n2', 120960, 1) | ID:8687 => ('n13', 1113600, 1)
i:15 => ('n10', 1305600, 1) | ID:9361 => ('n10', 399360, 1)
i:16 => ('n7', 1380480, 1) | ID:2547 => ('n5', 660480, 1)
i:17 => ('n13', 929280, 1) | ID:1455 => ('n3', 1693440, 1)
i:18 => ('n11', 293760, 1) | ID:8522 => ('n5', 360960, 1)
i:19 => ('n5', 1292160, 1) | ID:1260 => ('n7', 1612800, 1)
i:20 => ('n6', 735360, 1) | ID:7922 => ('n5', 1057920, 1)
i:21 => ('n14', 1413120, 1) | ID:1095 => ('n13', 1710720, 1)
i:22 => ('n14', 96000, 1) | ID:9216 => ('n11', 2002560, 1)
i:23 => ('n5', 1620480, 1) | ID:5261 => ('n6', 658560, 1)
i:24 => 

i:6 => ('n1', 138240, 1) | ID:72 => ('n1', 1463040, 1)
i:8 => ('n2', 1555200, 1) | ID:2975 => ('n14', 1355520, 1)
i:7 => ('n1', 1893120, 1) | ID:986 => ('n1', 1461120, 1)
i:9 => ('n13', 1476480, 1) | ID:5412 => ('n9', 979200, 1)
i:8 => ('n1', 1148160, 1) | ID:598 => ('n1', 579840, 1)
i:10 => ('n9', 1203840, 1) | ID:3620 => ('n13', 1403520, 1)
i:9 => ('n1', 501120, 1) | ID:261 => ('n1', 380160, 1)
i:11 => ('n11', 84480, 0) | ID:7622 => ('n9', 1568640, 1)
i:10 => ('n1', 1445760, 1) | ID:753 => ('n1', 2044800, 1)
i:12 => ('n14', 842880, 1) | ID:3547 => ('n11', 1234560, 1)
i:11 => ('n1', 1536000, 1) | ID:800 => ('n1', 1739520, 1)
i:13 => ('n3', 34560, 1) | ID:2350 => ('n3', 1428480, 0)
i:14 => ('n2', 120960, 1) | ID:3661 => ('n3', 1797120, 1)
i:12 => ('n1', 775680, 1) | ID:404 => ('n1', 1870080, 1)
i:15 => ('n10', 1305600, 1) | ID:1802 => ('n6', 1774080, 1)
i:13 => ('n1', 1376640, 1) | ID:717 => ('n1', 220800, 1)
i:16 => ('n7', 1380480, 1) | ID:1350 => ('n6', 1215360, 1)
i:14 => ('n1', 181

i:30 => ('n1', 1637760, 1) | ID:37 => ('n1', 147840, 1)
i:31 => ('n1', 913920, 1) | ID:915 => ('n1', 1207680, 1)
i:0 => ('n1', 1695360, 1) | ID:468 => ('n1', 1908480, 1)
i:1 => ('n1', 854400, 1) | ID:322 => ('n1', 366720, 1)
i:2 => ('n1', 326400, 1) | ID:237 => ('n1', 1785600, 1)
i:3 => ('n1', 476160, 1) | ID:288 => ('n1', 391680, 1)
i:4 => ('n1', 1522560, 1) | ID:888 => ('n1', 1397760, 1)
i:5 => ('n1', 1555200, 1) | ID:126 => ('n1', 289920, 1)
i:6 => ('n1', 138240, 1) | ID:225 => ('n1', 2136960, 1)
i:7 => ('n1', 1893120, 1) | ID:1073 => ('n1', 130560, 1)
i:8 => ('n1', 1148160, 1) | ID:353 => ('n1', 1776000, 1)
i:9 => ('n1', 501120, 1) | ID:858 => ('n1', 766080, 1)
i:10 => ('n1', 1445760, 1) | ID:243 => ('n1', 2048640, 1)
i:11 => ('n1', 1536000, 1) | ID:14 => ('n1', 1812480, 1)
i:12 => ('n1', 775680, 1) | ID:1076 => ('n1', 1182720, 1)
i:13 => ('n1', 1376640, 1) | ID:1042 => ('n1', 389760, 1)
i:14 => ('n1', 1812480, 1) | ID:1066 => ('n1', 1603200, 1)
i:15 => ('n1', 416640, 1) | ID:1085 

...

i:7 => ('n11', 890880, 1) | ID:5105 => ('n6', 1209600, 1)
i:8 => ('n9', 1920, 0) | ID:7370 => ('n10', 349440, 1)
i:9 => ('n2', 284160, 1) | ID:8386 => ('n9', 161280, 1)
i:10 => ('n9', 1553280, 1) | ID:994 => ('n5', 291840, 1)
i:11 => ('n3', 685440, 1) | ID:722 => ('n11', 170880, 1)
i:12 => ('n11', 1144320, 1) | ID:7977 => ('n5', 656640, 1)
i:13 => ('n7', 1361280, 1) | ID:5145 => ('n10', 1157760, 1)
i:14 => ('n10', 497280, 1) | ID:8757 => ('n9', 873600, 1)
i:15 => ('n13', 693120, 1) | ID:6783 => ('n10', 1593600, 1)
i:16 => ('n9', 207360, 1) | ID:9697 => ('n6', 1351680, 1)
i:17 => ('n3', 1668480, 1) | ID:3961 => ('n5', 526080, 1)
i:18 => ('n9', 1954560, 0) | ID:8145 => ('n13', 633600, 1)
i:19 => ('n9', 1804800, 0) | ID:2526 => ('n9', 232320, 1)
i:20 => ('n9', 1758720, 0) | ID:6911 => ('n11', 1052160, 1)
i:21 => ('n14', 428160, 1) | ID:6570 => ('n13', 297600, 1)
i:22 => ('n2', 268800, 1) | ID:5549 => ('n11', 188160, 1)
i:23 => ('n2', 1011840, 1) | ID:855 => ('n2', 86400, 1)
i:24 => ('n10'

i:15 => ('n13', 693120, 1) | ID:2886 => ('n11', 622080, 1)
i:16 => ('n9', 207360, 1) | ID:4968 => ('n7', 1080960, 1)
i:17 => ('n3', 1668480, 1) | ID:8481 => ('n13', 1434240, 1)
i:18 => ('n9', 1954560, 0) | ID:445 => ('n14', 1415040, 1)
i:19 => ('n9', 1804800, 0) | ID:2490 => ('n13', 864000, 1)
i:20 => ('n9', 1758720, 0) | ID:3371 => ('n5', 274560, 1)
i:21 => ('n14', 428160, 1) | ID:9610 => ('n11', 2002560, 1)
i:22 => ('n2', 268800, 1) | ID:6856 => ('n13', 879360, 1)
i:23 => ('n2', 1011840, 1) | ID:109 => ('n6', 1843200, 1)
i:24 => ('n10', 297600, 1) | ID:8723 => ('n3', 1633920, 1)
i:25 => ('n5', 1424640, 1) | ID:9720 => ('n9', 1545600, 1)
i:26 => ('n9', 579840, 1) | ID:5558 => ('n3', 1021440, 1)
i:27 => ('n13', 576000, 1) | ID:4542 => ('n2', 410880, 1)
i:28 => ('n5', 1280640, 1) | ID:3513 => ('n7', 967680, 1)
i:29 => ('n10', 1138560, 1) | ID:4969 => ('n2', 773760, 1)
i:30 => ('n2', 1109760, 1) | ID:7008 => ('n3', 679680, 1)
i:31 => ('n3', 161280, 1) | ID:7057 => ('n14', 1779840, 0)
i:0

i:7 => ('n11', 890880, 1) | ID:1135 => ('n11', 597120, 1)
i:8 => ('n9', 1920, 0) | ID:935 => ('n6', 117120, 1)
i:16 => ('n1', 1781760, 1) | ID:928 => ('n1', 1850880, 0)
i:17 => ('n1', 1040640, 1) | ID:542 => ('n1', 1568640, 1)
i:9 => ('n2', 284160, 1) | ID:2143 => ('n6', 881280, 1)
i:18 => ('n1', 138240, 1) | ID:72 => ('n1', 1125120, 1)
i:10 => ('n9', 1553280, 1) | ID:8649 => ('n2', 1572480, 1)
i:11 => ('n3', 685440, 1) | ID:9442 => ('n6', 704640, 1)
i:19 => ('n1', 1065600, 1) | ID:555 => ('n1', 2115840, 1)
i:12 => ('n11', 1144320, 1) | ID:5360 => ('n14', 1559040, 1)
i:20 => ('n1', 328320, 1) | ID:171 => ('n1', 1392000, 1)
i:21 => ('n1', 577920, 1) | ID:301 => ('n1', 1234560, 1)
i:13 => ('n7', 1361280, 1) | ID:9695 => ('n11', 393600, 1)
i:22 => ('n1', 526080, 1) | ID:274 => ('n1', 1537920, 1)
i:23 => ('n1', 1939200, 1) | ID:1010 => ('n1', 1418880, 1)
i:14 => ('n10', 497280, 1) | ID:621 => ('n5', 1528320, 1)
i:24 => ('n1', 1689600, 1) | ID:880 => ('n1', 2123520, 1)
i:15 => ('n13', 69312

i:8 => ('n1', 55680, 1) | ID:1045 => ('n1', 1027200, 1)
i:9 => ('n1', 2138880, 1) | ID:661 => ('n1', 460800, 1)
i:10 => ('n1', 412800, 1) | ID:663 => ('n1', 923520, 1)
i:11 => ('n1', 2071680, 1) | ID:576 => ('n1', 979200, 1)
i:12 => ('n1', 2060160, 1) | ID:262 => ('n1', 1843200, 1)
i:13 => ('n1', 1743360, 1) | ID:152 => ('n1', 1701120, 1)
i:14 => ('n1', 2140800, 1) | ID:209 => ('n1', 1382400, 1)
i:15 => ('n1', 1647360, 1) | ID:679 => ('n1', 147840, 1)
i:16 => ('n1', 1781760, 1) | ID:846 => ('n1', 115200, 1)
i:17 => ('n1', 1040640, 1) | ID:136 => ('n1', 2006400, 1)
i:18 => ('n1', 138240, 1) | ID:1105 => ('n1', 455040, 1)
i:19 => ('n1', 1065600, 1) | ID:499 => ('n1', 493440, 1)
i:20 => ('n1', 328320, 1) | ID:693 => ('n1', 88320, 1)
i:21 => ('n1', 577920, 1) | ID:439 => ('n1', 1672320, 1)
i:22 => ('n1', 526080, 1) | ID:198 => ('n1', 844800, 1)
i:23 => ('n1', 1939200, 1) | ID:219 => ('n1', 1541760, 1)
i:24 => ('n1', 1689600, 1) | ID:886 => ('n1', 819840, 1)
i:25 => ('n1', 576000, 1) | ID:5

i:16 => ('n2', 1896960, 0) | ID:7492 => ('n5', 662400, 1)
i:17 => ('n9', 15360, 0) | ID:4351 => ('n3', 1138560, 1)
i:18 => ('n6', 1670400, 1) | ID:8834 => ('n11', 1296000, 1)
i:19 => ('n5', 130560, 1) | ID:236 => ('n13', 888960, 1)
i:20 => ('n7', 57600, 0) | ID:5873 => ('n9', 1668480, 1)
i:21 => ('n5', 1128960, 1) | ID:3742 => ('n3', 1816320, 1)


In [8]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

names = ["n1", "n2", "n3"]
np_data_dir = "G:/cap-sleep-database-1.0.0/cap-sleep-database-1.0.0/np_data_64"

def leave_ones():
    metrics_df = pd.DataFrame(columns=['validation_sub', 'loss', 'accuracy', 'val_loss', 'val_accuracy', 'time_per_epoch'])
    
    for i in range(len(names)):
        val = names[i]
        train = names[i+1:] + names[:i]
        
        print(val, train)
        print("==========================================================")
        checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
        early = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=1, mode='auto')
        training_generator = DataGenerator(np_data_dir, list_rec_names=train)
        print("Number of batches for training generator:", len(training_generator))
        print("==========================================================")
        validation_generator = DataGenerator(np_data_dir, list_rec_names=[val])
        print("Number of batches for validation generator:", len(validation_generator))
        
        start_time = time.time()
        hist = model.fit_generator(steps_per_epoch=5, generator=training_generator, validation_data=validation_generator, validation_steps=5, epochs=10, callbacks=[checkpoint, early])
        end_time = time.time()
        
        iteration_name = f"Subject-{val}"
        hist_df = pd.DataFrame(hist.history)
        hist_df['validation_sub'] = iteration_name
        
        time_per_epoch = (end_time - start_time) / len(hist.history['loss'])
        hist_df['time_per_epoch'] = time_per_epoch
        
        metrics_df = metrics_df.append(hist_df, ignore_index=True)
        
        # Save the metrics DataFrame to Excel after each iteration (epoch)
        metrics_df.to_excel('metrics.xlsx', index=False)
        
        # Load the trained model from the saved .h5 file
        model_val = load_model("vgg16_1.h5")
        
        # Create a generator for the validation data of the current subject
        val_generator = DataGenerator(np_data_dir, list_rec_names=[val])
        
        # Make predictions on the validation data using the loaded model
        predictions = model_val.predict_generator(val_generator)
        
        # Get the actual labels from the generator
        actual_labels = np.concatenate([val_generator[i][1] for i in range(len(val_generator))])
        
        # Create a DataFrame to store the actual and predicted values
        prediction_df = pd.DataFrame({'Actual': actual_labels[:, 0], 'Predicted': predictions[:, 0]})
        
        # Save the DataFrame to a CSV file with the subject name
        prediction_df.to_csv(f'{val}_predictions.csv', index=False)


In [9]:
leave_ones()

n1 ['n2', 'n3']
Initialise meta data of samples
Loading n2...
[n2] (2104320,), n_seg:1096
recording:(2104320,)
...[n2], n_seg:999 out of 1096 (lags seg:97, samples:186240), bad_label: 0, bad_len/ignored: 0, remaining:186240
Loading n3...
[n3] (1921984,), n_seg:1001
recording:(1921984,)
...[n3], n_seg:999 out of 1001 (lags seg:2, samples:3840), bad_label: 0, bad_len/ignored: 0, remaining:3904
Total sample-meta loaded: 1998
Number of batches for training generator: 62
Initialise meta data of samples
Loading n1...
[n1] (2202240,), n_seg:1147
recording:(2202240,)
...[n1], n_seg:1140 out of 1147 (lags seg:7, samples:13440), bad_label: 0, bad_len/ignored: 0, remaining:13440
Total sample-meta loaded: 1140
Number of batches for validation generator: 35
i:0 => ('n2', 708480, 1) | ID:369 => ('n3', 142080, 0)
i:1 => ('n3', 57600, 1) | ID:1029 => ('n2', 220800, 1)
i:2 => ('n2', 1351680, 1) | ID:704 => ('n2', 545280, 1)
i:3 => ('n3', 432000, 1) | ID:1224 => ('n3', 74880, 1)
i:4 => ('n2', 0, 0) | ID

i:28 => ('n3', 958080, 1) | ID:54 => ('n2', 1153920, 1)
i:29 => ('n3', 439680, 1) | ID:840 => ('n2', 503040, 1)
i:30 => ('n2', 979200, 1) | ID:583 => ('n2', 1758720, 1)
i:31 => ('n2', 1670400, 1) | ID:258 => ('n2', 695040, 1)
i:0 => ('n2', 708480, 1) | ID:1700 => ('n3', 1418880, 1)
3/5 [=================>............] - ETA: 9:47 - loss: 0.5668 - accuracy: 0.7083 i:1 => ('n3', 57600, 1) | ID:1491 => ('n2', 1388160, 1)
i:2 => ('n2', 1351680, 1) | ID:1507 => ('n2', 639360, 0)
i:3 => ('n3', 432000, 1) | ID:265 => ('n2', 654720, 1)
i:4 => ('n2', 0, 0) | ID:1446 => ('n2', 919680, 1)
i:5 => ('n3', 1372800, 1) | ID:1235 => ('n2', 625920, 1)
i:6 => ('n3', 1493760, 1) | ID:563 => ('n3', 1908480, 0)
i:7 => ('n3', 278400, 1) | ID:1320 => ('n3', 1774080, 1)
i:8 => ('n3', 691200, 1) | ID:1730 => ('n3', 44160, 1)
i:9 => ('n3', 255360, 1) | ID:1404 => ('n2', 234240, 1)
i:10 => ('n3', 376320, 1) | ID:1293 => ('n2', 337920, 1)
i:11 => ('n3', 1747200, 1) | ID:1977 => ('n3', 366720, 1)
i:12 => ('n3', 670

i:29 => ('n3', 439680, 1) | ID:923 => ('n3', 1712640, 1)
i:1 => ('n1', 1877760, 1) | ID:402 => ('n1', 441600, 1)
i:30 => ('n2', 979200, 1) | ID:1386 => ('n2', 161280, 1)
i:2 => ('n1', 491520, 1) | ID:591 => ('n1', 794880, 1)
i:31 => ('n2', 1670400, 1) | ID:1478 => ('n3', 1806720, 1)
i:3 => ('n1', 708480, 1) | ID:269 => ('n1', 633600, 1)
i:4 => ('n1', 1910400, 1) | ID:654 => ('n1', 464640, 0)
i:5 => ('n1', 280320, 1) | ID:634 => ('n1', 2021760, 1)
i:6 => ('n1', 1695360, 1) | ID:428 => ('n1', 1804800, 1)
i:7 => ('n1', 1906560, 1) | ID:610 => ('n1', 1228800, 1)
i:8 => ('n1', 1361280, 1) | ID:187 => ('n1', 407040, 1)
i:9 => ('n1', 288000, 1) | ID:805 => ('n1', 1292160, 1)
i:10 => ('n1', 529920, 1) | ID:218 => ('n1', 480000, 1)
i:11 => ('n1', 201600, 1) | ID:58 => ('n1', 426240, 1)
i:12 => ('n1', 1553280, 1) | ID:1070 => ('n1', 38400, 1)
i:13 => ('n1', 593280, 1) | ID:988 => ('n1', 2033280, 1)
i:14 => ('n1', 1063680, 1) | ID:739 => ('n1', 702720, 1)
i:15 => ('n1', 1908480, 1) | ID:417 => ('

i:15 => ('n1', 2006400, 1) | ID:717 => ('n1', 1054080, 1)
i:16 => ('n1', 1918080, 1) | ID:718 => ('n1', 2171520, 1)
i:17 => ('n1', 1512960, 1) | ID:462 => ('n1', 170880, 1)
i:18 => ('n1', 2075520, 1) | ID:863 => ('n1', 7680, 0)
i:19 => ('n1', 205440, 1) | ID:337 => ('n1', 1376640, 1)
i:20 => ('n1', 766080, 1) | ID:862 => ('n1', 798720, 1)
i:21 => ('n1', 67200, 1) | ID:650 => ('n1', 2158080, 1)
i:22 => ('n1', 1797120, 1) | ID:186 => ('n1', 1666560, 1)
i:23 => ('n1', 1130880, 1) | ID:128 => ('n1', 625920, 1)
i:24 => ('n1', 1645440, 1) | ID:360 => ('n1', 316800, 1)
i:25 => ('n1', 1749120, 1) | ID:919 => ('n1', 1839360, 1)
i:26 => ('n1', 816000, 1) | ID:408 => ('n1', 1021440, 1)
i:27 => ('n1', 1094400, 1) | ID:212 => ('n1', 1656960, 1)
i:28 => ('n1', 1121280, 0) | ID:112 => ('n1', 180480, 1)
i:29 => ('n1', 593280, 1) | ID:577 => ('n1', 1213440, 1)
i:30 => ('n1', 1660800, 1) | ID:751 => ('n1', 1355520, 1)
i:31 => ('n1', 1413120, 1) | ID:716 => ('n1', 142080, 1)

Epoch 1: val_loss improved f

i:23 => ('n2', 1512960, 1) | ID:1285 => ('n2', 635520, 1)
i:24 => ('n2', 257280, 1) | ID:1742 => ('n2', 1733760, 1)
i:25 => ('n3', 1290240, 0) | ID:274 => ('n3', 63360, 1)
i:26 => ('n2', 113280, 1) | ID:409 => ('n2', 1428480, 1)
i:27 => ('n3', 729600, 1) | ID:490 => ('n2', 1386240, 1)
i:28 => ('n2', 1009920, 1) | ID:1689 => ('n2', 1390080, 1)
i:29 => ('n2', 1269120, 0) | ID:458 => ('n3', 796800, 1)
i:30 => ('n2', 1111680, 1) | ID:1427 => ('n2', 303360, 1)
i:31 => ('n2', 1008000, 1) | ID:1823 => ('n3', 1739520, 1)
i:0 => ('n2', 610560, 1) | ID:1996 => ('n2', 1804800, 0)
4/5 [=======================>......] - ETA: 3:41 - loss: 0.3184 - accuracy: 0.9062i:1 => ('n3', 1213440, 1) | ID:561 => ('n2', 370560, 1)
i:2 => ('n3', 1098240, 1) | ID:368 => ('n2', 451200, 1)
i:3 => ('n2', 1034880, 1) | ID:1657 => ('n3', 842880, 1)
i:4 => ('n2', 1877760, 0) | ID:332 => ('n2', 1230720, 1)
i:5 => ('n3', 1866240, 1) | ID:1822 => ('n3', 906240, 1)
i:6 => ('n3', 1564800, 1) | ID:1039 => ('n3', 610560, 1)
i:

...

i:19 => ('n3', 1155840, 1) | ID:67 => ('n3', 1516800, 1)
i:20 => ('n3', 823680, 1) | ID:260 => ('n3', 1802880, 1)
i:21 => ('n3', 439680, 1) | ID:298 => ('n3', 1436160, 1)
i:22 => ('n3', 1883520, 1) | ID:241 => ('n3', 1589760, 1)
i:23 => ('n3', 76800, 1) | ID:719 => ('n3', 624000, 1)
i:24 => ('n3', 1048320, 1) | ID:925 => ('n3', 1489920, 1)
i:25 => ('n3', 1105920, 1) | ID:578 => ('n3', 1299840, 0)
i:26 => ('n3', 549120, 1) | ID:340 => ('n3', 1716480, 1)
i:27 => ('n3', 577920, 1) | ID:111 => ('n3', 282240, 1)
i:28 => ('n3', 1509120, 1) | ID:476 => ('n3', 1459200, 1)
i:29 => ('n3', 1357440, 1) | ID:758 => ('n3', 1852800, 1)
i:30 => ('n3', 1470720, 1) | ID:792 => ('n3', 1272960, 0)
i:31 => ('n3', 1912320, 0) | ID:849 => ('n3', 695040, 1)
i:0 => ('n3', 748800, 1) | ID:517 => ('n3', 919680, 1)
i:1 => ('n3', 1712640, 1) | ID:141 => ('n3', 1365120, 0)
i:2 => ('n3', 1564800, 1) | ID:518 => ('n3', 1639680, 1)
i:3 => ('n3', 1261440, 0) | ID:278 => ('n3', 1862400, 1)
i:4 => ('n3', 537600, 1) | ID:

i:4 => ('n3', 537600, 1) | ID:266 => ('n3', 620160, 1)
i:5 => ('n3', 737280, 1) | ID:604 => ('n3', 1288320, 0)
i:6 => ('n3', 654720, 1) | ID:326 => ('n3', 1255680, 0)
i:7 => ('n3', 481920, 1) | ID:92 => ('n3', 975360, 1)
i:8 => ('n3', 1176960, 1) | ID:366 => ('n3', 57600, 1)
i:9 => ('n3', 295680, 1) | ID:588 => ('n3', 616320, 1)
i:10 => ('n3', 115200, 0) | ID:710 => ('n3', 1209600, 1)
i:11 => ('n3', 1478400, 0) | ID:405 => ('n3', 122880, 0)
i:12 => ('n3', 1121280, 1) | ID:703 => ('n3', 1630080, 1)
i:13 => ('n3', 1733760, 1) | ID:734 => ('n3', 355200, 1)
i:14 => ('n3', 1384320, 1) | ID:205 => ('n3', 633600, 1)
i:15 => ('n3', 1280640, 0) | ID:830 => ('n3', 263040, 1)
i:16 => ('n3', 504960, 1) | ID:468 => ('n3', 1766400, 1)
i:17 => ('n3', 28800, 0) | ID:362 => ('n3', 877440, 0)
i:18 => ('n3', 1161600, 1) | ID:264 => ('n3', 1875840, 1)
i:19 => ('n3', 1155840, 1) | ID:484 => ('n3', 136320, 0)
i:20 => ('n3', 823680, 1) | ID:279 => ('n3', 1361280, 1)
i:21 => ('n3', 439680, 1) | ID:442 => ('n3

i:22 => ('n3', 1883520, 1) | ID:427 => ('n3', 376320, 1)
i:23 => ('n3', 76800, 1) | ID:995 => ('n3', 433920, 1)
i:24 => ('n3', 1048320, 1) | ID:510 => ('n3', 1728000, 1)
i:25 => ('n3', 1105920, 1) | ID:351 => ('n3', 1904640, 1)
i:26 => ('n3', 549120, 1) | ID:320 => ('n3', 1530240, 1)
i:27 => ('n3', 577920, 1) | ID:948 => ('n3', 600960, 1)
i:28 => ('n3', 1509120, 1) | ID:629 => ('n3', 19200, 0)
i:29 => ('n3', 1357440, 1) | ID:2 => ('n3', 1564800, 1)
i:30 => ('n3', 1470720, 1) | ID:950 => ('n3', 1036800, 1)
i:31 => ('n3', 1912320, 0) | ID:725 => ('n3', 51840, 1)
i:0 => ('n3', 5760, 1) | ID:3 => ('n3', 145920, 0)
i:1 => ('n3', 1315200, 0) | ID:685 => ('n3', 504960, 1)
i:2 => ('n3', 1576320, 1) | ID:821 => ('n3', 253440, 1)
i:3 => ('n3', 145920, 0) | ID:76 => ('n3', 1637760, 1)
i:4 => ('n3', 647040, 1) | ID:337 => ('n3', 1524480, 1)
i:5 => ('n3', 1057920, 1) | ID:551 => ('n3', 1635840, 1)
i:6 => ('n3', 1165440, 1) | ID:607 => ('n3', 1200000, 1)
i:7 => ('n3', 424320, 1) | ID:221 => ('n3', 6

i:15 => ('n3', 564480, 1) | ID:668 => ('n3', 40320, 1)
i:16 => ('n3', 11520, 1) | ID:158 => ('n3', 44160, 1)
i:17 => ('n3', 1520640, 1) | ID:87 => ('n3', 912000, 1)
i:18 => ('n3', 282240, 1) | ID:926 => ('n3', 1585920, 1)
i:19 => ('n3', 201600, 1) | ID:521 => ('n3', 1860480, 1)
i:20 => ('n3', 835200, 1) | ID:185 => ('n3', 274560, 1)
i:21 => ('n3', 1234560, 1) | ID:436 => ('n3', 1340160, 0)
i:22 => ('n3', 1850880, 1) | ID:743 => ('n3', 804480, 1)
i:23 => ('n3', 929280, 1) | ID:864 => ('n3', 1681920, 1)
i:24 => ('n3', 892800, 0) | ID:703 => ('n3', 21120, 0)
i:25 => ('n3', 466560, 1) | ID:406 => ('n3', 107520, 0)
i:26 => ('n3', 814080, 1) | ID:219 => ('n3', 218880, 1)
i:27 => ('n3', 376320, 1) | ID:114 => ('n3', 579840, 1)
i:28 => ('n3', 1685760, 1) | ID:660 => ('n3', 958080, 1)
i:29 => ('n3', 263040, 1) | ID:36 => ('n3', 812160, 1)
i:30 => ('n3', 675840, 1) | ID:191 => ('n3', 1223040, 1)
i:31 => ('n3', 1002240, 1) | ID:613 => ('n3', 917760, 1)
i:0 => ('n3', 5760, 1) | ID:747 => ('n3', 76

i:27 => ('n3', 376320, 1) | ID:199 => ('n3', 251520, 1)
i:28 => ('n3', 1685760, 1) | ID:933 => ('n3', 374400, 1)
i:29 => ('n3', 263040, 1) | ID:818 => ('n3', 1495680, 1)
i:30 => ('n3', 675840, 1) | ID:154 => ('n3', 1478400, 0)
i:31 => ('n3', 1002240, 1) | ID:485 => ('n3', 3840, 1)
i:0 => ('n3', 5760, 1) | ID:457 => ('n3', 23040, 0)
i:1 => ('n3', 1315200, 0) | ID:841 => ('n3', 1547520, 1)
i:2 => ('n3', 1576320, 1) | ID:441 => ('n3', 1848960, 1)
i:3 => ('n3', 145920, 0) | ID:575 => ('n3', 1653120, 1)
i:4 => ('n3', 647040, 1) | ID:858 => ('n3', 1680000, 1)
i:5 => ('n3', 1057920, 1) | ID:836 => ('n3', 1211520, 1)
i:6 => ('n3', 1165440, 1) | ID:90 => ('n3', 1443840, 1)
i:7 => ('n3', 424320, 1) | ID:639 => ('n3', 1142400, 1)
i:8 => ('n3', 1537920, 1) | ID:80 => ('n3', 1683840, 1)
i:9 => ('n3', 910080, 1) | ID:835 => ('n3', 1668480, 1)
i:10 => ('n3', 969600, 1) | ID:935 => ('n3', 1612800, 1)
i:11 => ('n3', 1054080, 1) | ID:174 => ('n3', 36480, 1)
i:12 => ('n3', 86400, 1) | ID:581 => ('n3', 13